### Netflix Prize Dataset EDA

In this notebook, we will primarily be performing some exploratory data analysis on the Netflix Prize dataset.

Our initial questions that we want to explore with this data set:
1. What are the most rated movies?
2. What are the most POORLY rated movies?
3. Who are these “top 10k” users?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime

plt.style.use('seaborn')
sns.set_palette("husl")

In [ ]:


# Load the data
def load_data(ratings_file, movies_file):
    """
    Load and prepare Netflix Prize dataset
    ratings_file: path to ratings.csv
    movies_file: path to movie_titles.csv
    """
    # Load ratings data
    # Format: MovieID,UserID,Rating,Date
    ratings_df = pd.read_csv(ratings_file, names=['MovieID', 'UserID', 'Rating', 'Date'])
    
    # Load movie titles data
    # Format: MovieID,Year,Title
    movies_df = pd.read_csv(movies_file, names=['MovieID', 'Year', 'Title'], encoding='latin-1')
    
    return ratings_df, movies_df
